## Import thư viện

In [ ]:

import numpy as np
import  pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook as tqdm
import seaborn as sn
sn.set()

import sys
import warnings; 
warnings.simplefilter('ignore')

,Unnamed: 0,userId,movieId,rating,timestamp,sid,uid
0,6,1,151,4.0,1094785734,1210,0
1,7,1,223,4.0,1112485573,47,0
2,8,1,253,4.0,1112484940,308,0
3,9,1,260,4.0,1112484826,4,0
4,10,1,293,4.0,1112484703,574,0


## Download data

In [ ]:
! rm *.zip
! rm -rf data

!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip ml-20m.zip
!mkdir data
!mv ml-20m data

rm: cannot remove '*.zip': No such file or directory
--2021-04-06 14:36:47--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  26.9MB/s    in 7.8s    

2021-04-06 14:36:55 (24.4 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


## Xử lý dữ liệu

In [ ]:
### change `DATA_DIR` to the location where movielens-20m dataset sits
DATA_DIR = './data/ml-20m/'

In [ ]:
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0)

In [ ]:
raw_data = raw_data[raw_data['rating'] > 3.5]
raw_data

,userId,movieId,rating,timestamp
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826
10,1,293,4.0,1112484703
...,...,...,...,...
20000256,138493,66762,4.5,1255805408
20000257,138493,68319,4.5,1260209720
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108


In [ ]:
def filter_data(data,key):
  # chỉ giữ lại những user vote ít nhất 5 items
  df = data.copy()
  count_movie_on_user =  df.groupby(key).count()
  user_id = count_movie_on_user[count_movie_on_user.movieId>= 5].index
  filtered = df[df[key].isin(user_id)]
  return filtered


In [ ]:
# filtered = filter_data(raw_data,'userId')

In [ ]:
list_user = raw_data.userId.unique()
# CHia data thành các tập train test và validation theo train test split
# train_user, test_user = train_test_split(list_user, test_size=0.2, random_state=42)
# train_user, val_user =  train_test_split(train_user, test_size=0.2, random_state=41)


# 10,000 user để đánh giá 
np.random.seed(98765)
idx_perm = np.random.permutation(len(list_user))
list_user = list_user[idx_perm]

n_heldout_users = 10_000

train_user = list_user[:-n_heldout_users * 2]
val_user = list_user[- n_heldout_users :]
test_user = list_user[ -2*n_heldout_users: -n_heldout_users]

In [ ]:
# Chia thành các tập train, test, validation từ tập user đã được chia
train_data = raw_data[raw_data.userId.isin(train_user)].copy()
val_data = raw_data[raw_data.userId.isin(val_user)].copy()
test_data = raw_data[raw_data.userId.isin(test_user)].copy()

# chỉ giữ lại những user đã vote ít nhất 5 movies
train_data = filter_data(train_data,'userId')
val_data = filter_data(val_data,'userId')
test_data = filter_data(test_data,'userId')


In [ ]:
# Từ tập train lấy ra danh sách id của tập train, tập validation và tập test chỉ giữ những records có danh sách phim trong tập train.
unique_sid_train = train_data.movieId.unique()
val_data = val_data[val_data.movieId.isin(unique_sid_train)]
test_data = test_data[test_data.movieId.isin(unique_sid_train)]


# Gán lại id cho danh sách movies
show2id = dict((sid, i) for (i, sid) in enumerate(train_data.movieId.unique()))

train_data['sid'] = train_data.movieId.apply(lambda x:show2id[x])
test_data['sid'] = test_data.movieId.apply(lambda x:show2id[x])
val_data['sid'] = val_data.movieId.apply(lambda x:show2id[x])



In [ ]:
# gán lại id cho danh sách user 
train_uid = dict((uid, i) for (i, uid) in enumerate(train_data.userId.unique()))
val_uid = dict((uid, i) for (i, uid) in enumerate(val_data.userId.unique()))
test_uid = dict((uid, i) for (i, uid) in enumerate(test_data.userId.unique()))

train_data['uid'] = train_data.userId.apply(lambda x:train_uid[x])
val_data['uid'] = val_data.userId.apply(lambda x:val_uid[x])
test_data['uid'] = test_data.userId.apply(lambda x:test_uid[x])


In [ ]:
train_data.uid.nunique(),val_data.uid.nunique(),test_data.uid.nunique()

(116907, 9895, 9875)

# Save data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/data/ml-20m/'

In [ ]:
train_data.to_csv(data_path + 'train.csv',index=False)
val_data.to_csv(data_path + 'val.csv',index=False)
test_data.to_csv(data_path + 'test.csv',index=False)

In [ ]:
train_data = pd.read_csv(data_path+'train.csv')
val_data = pd.read_csv(data_path+'val.csv')
test_data = pd.read_csv(data_path+'test.csv')